### Loading packages

In [1]:
import os
import sys
sys.path.insert(0, './deeplab_v3/')
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import numpy as np
import network
slim = tf.contrib.slim
import argparse
import json
from preprocessing.read_data import tf_record_parser, scale_image_with_crop_padding
from preprocessing import training
from metrics import *

### Plot pretty figures

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving the figures in Figs folder (If this folder does not exist already, this function will make Figs directory)
FIGS_DIR = "processed_images"
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=150):
    if not os.path.isdir(FIGS_DIR):
        os.makedirs(FIGS_DIR)
    path = os.path.join(".",FIGS_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, bbox_inches='tight', format=fig_extension, dpi=resolution)

### Selecting pre-trained model for evaluation 

In [3]:
plt.interactive(False)
parser = argparse.ArgumentParser()
envarg = parser.add_argument_group('Eval params')
envarg.add_argument("--model_id", default=16645, type=int, help="Model id name to be loaded.")
log_folder = './deeplab_v3/tboard_logs'
model_name = '16645'
if not os.path.exists(os.path.join(log_folder, model_name, "test")):
    os.makedirs(os.path.join(log_folder, model_name, "test"))

with open(log_folder + '/' + model_name + '/train/data.json', 'r') as fp:
    args = json.load(fp)

### Setting up different category of objects in deeplab
We just care about cars and one final touch that we can do is just keeping the object that are detectd as car --> category number 7

In [4]:
class Dotdict(dict):
     """dot.notation access to dictionary attributes"""
     __getattr__ = dict.get
     __setattr__ = dict.__setitem__
     __delattr__ = dict.__delitem__

args = Dotdict(args)

# 0=background
# 1=aeroplane
# 2=bicycle
# 3=bird
# 4=boat
# 5=bottle
# 6=bus
# 7=car
# 8=cat
# 9=chair
# 10=cow
# 11=diningtable
# 12=dog
# 13=horse
# 14=motorbike
# 15=person
# 16=potted plant
# 17=sheep
# 18=sofa
# 19=train
# 20=tv/monitor
# 255=unknown

class_labels = [v for v in range((args.number_of_classes+1))]
class_labels[-1] = 255


### Providing the directory and the filename for image dataset in TFRecords format

In [5]:
LOG_FOLDER = './deeplab_v3/tboard_logs'
TEST_DATASET_DIR="./deeplab_v3/dataset/tfrecords"
TEST_FILE = 'test.tfrecords'

test_filenames = [os.path.join(TEST_DATASET_DIR,TEST_FILE)]
test_dataset = tf.data.TFRecordDataset(test_filenames)
test_dataset = test_dataset.map(tf_record_parser)  # Parse the record into tensors.
test_dataset = test_dataset.map(lambda image, annotation, image_shape: scale_image_with_crop_padding(image, annotation, image_shape, args.crop_size))
test_dataset = test_dataset.shuffle(buffer_size=100)
test_dataset = test_dataset.batch(args.batch_size)

iterator = test_dataset.make_one_shot_iterator()
batch_images_tf, batch_labels_tf, batch_shapes_tf = iterator.get_next()

logits_tf =  network.deeplab_v3(batch_images_tf, args, is_training=False, reuse=False)

valid_labels_batch_tf, valid_logits_batch_tf = training.get_valid_logits_and_labels(
    annotation_batch_tensor=batch_labels_tf,
    logits_batch_tensor=logits_tf,
    class_labels=class_labels)

cross_entropies_tf = tf.nn.softmax_cross_entropy_with_logits_v2(logits=valid_logits_batch_tf,
                                                                labels=valid_labels_batch_tf)

cross_entropy_mean_tf = tf.reduce_mean(cross_entropies_tf)
tf.summary.scalar('cross_entropy', cross_entropy_mean_tf)

predictions_tf = tf.argmax(logits_tf, axis=3)
probabilities_tf = tf.nn.softmax(logits_tf)

merged_summary_op = tf.summary.merge_all()
saver = tf.train.Saver()

test_folder = os.path.join(log_folder, model_name, "test")
train_folder = os.path.join(log_folder, model_name, "train")

### Evaluating the pretrained model on the input dataset

In [6]:
with tf.Session() as sess:

    # Create a saver.
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    # Restore variables from disk.
    saver.restore(sess, os.path.join(train_folder, "model.ckpt"))
    print("Model", model_name, "restored.")

    mean_IoU = []
    mean_pixel_acc = []
    mean_freq_weighted_IU = []
    mean_acc = []

    cnt = 0
    while True:
        try:
            batch_images_np, batch_predictions_np, batch_labels_np, batch_shapes_np, summary_string= sess.run([batch_images_tf, predictions_tf, batch_labels_tf, batch_shapes_tf, merged_summary_op])
            heights, widths = batch_shapes_np

            # loop through the images in the batch and extract the valid areas from the tensors
            for i in range(batch_predictions_np.shape[0]):

                label_image = batch_labels_np[i]
                pred_image = batch_predictions_np[i]
                input_image = batch_images_np[i]

                indices = np.where(label_image != 255)
                label_image = label_image[indices]
                pred_image = pred_image[indices]
                input_image = input_image[indices]
                
                print(label_image.shape)

                if label_image.shape[0] == 262144:
                    label_image = np.reshape(label_image, (512,512))
                    pred_image = np.reshape(pred_image, (512,512))
                    input_image = np.reshape(input_image, (512,512,3))
                else:
                    label_image = np.reshape(label_image, (heights[i], widths[i]))
                    pred_image = np.reshape(pred_image, (heights[i], widths[i]))
                    input_image = np.reshape(input_image, (heights[i], widths[i], 3))

                pix_acc = pixel_accuracy(pred_image, label_image)
                m_acc = mean_accuracy(pred_image, label_image)
                IoU = mean_IU(pred_image, label_image)
                freq_weighted_IU = frequency_weighted_IU(pred_image, label_image)

                mean_pixel_acc.append(pix_acc)
                mean_acc.append(m_acc)
                mean_IoU.append(IoU)
                mean_freq_weighted_IU.append(freq_weighted_IU)

                f, (ax1, ax3) = plt.subplots(1, 2, figsize=(25, 25))

                ax1.imshow(input_image.astype(np.uint8))
                ax1.axis('off')
#                 ax2.imshow(input_image.astype(np.uint8))
#                 ax2.imshow(np.isin(pred_image, 7).astype(np.uint8), alpha=0.7)
#                             
                ax3.imshow(input_image.astype(np.uint8))
                ax3.imshow(pred_image, alpha=0.7) 
                ax3.axis('off')
        
                save_fig(str(cnt))
#                 plt.show()                
                plt.close()
                cnt += 1

        except tf.errors.OutOfRangeError:
            break

INFO:tensorflow:Restoring parameters from ./deeplab_v3/tboard_logs/16645/train/model.ckpt
Model 16645 restored.
(204288,)
Saving figure 0
(204288,)
Saving figure 1
(204288,)
Saving figure 2
(147456,)
Saving figure 3
(204288,)
Saving figure 4
(147456,)
Saving figure 5
(204288,)
Saving figure 6
(147456,)
Saving figure 7
(147456,)
Saving figure 8
(204288,)
Saving figure 9
(204288,)
Saving figure 10
(147456,)
Saving figure 11
(204288,)
Saving figure 12
(147456,)
Saving figure 13
(147456,)
Saving figure 14
(147456,)
Saving figure 15
(204288,)
Saving figure 16
(204288,)
Saving figure 17
(204288,)
Saving figure 18
(204288,)
Saving figure 19
(204288,)
Saving figure 20
(204288,)
Saving figure 21
(204288,)
Saving figure 22
(204288,)
Saving figure 23
(204288,)
Saving figure 24
(204288,)
Saving figure 25
(204288,)
Saving figure 26
(204288,)
Saving figure 27
(147456,)
Saving figure 28
(204288,)
Saving figure 29
(204288,)
Saving figure 30
(147456,)
Saving figure 31
(204288,)
Saving figure 32
(204288

(147456,)
Saving figure 293
(204288,)
Saving figure 294
(204288,)
Saving figure 295
(204288,)
Saving figure 296
(204288,)
Saving figure 297
(147456,)
Saving figure 298
(147456,)
Saving figure 299
(147456,)
Saving figure 300
(204288,)
Saving figure 301
(204288,)
Saving figure 302
(147456,)
Saving figure 303
(147456,)
Saving figure 304
(204288,)
Saving figure 305
(204288,)
Saving figure 306
(147456,)
Saving figure 307
(204288,)
Saving figure 308
(204288,)
Saving figure 309
(147456,)
Saving figure 310
(204288,)
Saving figure 311
(204288,)
Saving figure 312
(204288,)
Saving figure 313
(204288,)
Saving figure 314
(204288,)
Saving figure 315
(204288,)
Saving figure 316
(147456,)
Saving figure 317
(147456,)
Saving figure 318
(147456,)
Saving figure 319
(204288,)
Saving figure 320
(204288,)
Saving figure 321
(204288,)
Saving figure 322
(147456,)
Saving figure 323
(204288,)
Saving figure 324
(204288,)
Saving figure 325
(204288,)
Saving figure 326
(147456,)
Saving figure 327
(204288,)
Saving fig

(204288,)
Saving figure 586
(147456,)
Saving figure 587
(147456,)
Saving figure 588
(204288,)
Saving figure 589
(204288,)
Saving figure 590
(204288,)
Saving figure 591
(204288,)
Saving figure 592
(147456,)
Saving figure 593
(147456,)
Saving figure 594
(147456,)
Saving figure 595
(147456,)
Saving figure 596
(204288,)
Saving figure 597
(204288,)
Saving figure 598
(147456,)
Saving figure 599
(147456,)
Saving figure 600
(204288,)
Saving figure 601
(204288,)
Saving figure 602
(147456,)
Saving figure 603
(204288,)
Saving figure 604
(204288,)
Saving figure 605
(204288,)
Saving figure 606
(204288,)
Saving figure 607
(147456,)
Saving figure 608
(147456,)
Saving figure 609
(147456,)
Saving figure 610
(147456,)
Saving figure 611
(204288,)
Saving figure 612
(204288,)
Saving figure 613
(147456,)
Saving figure 614
(204288,)
Saving figure 615
(147456,)
Saving figure 616
(204288,)
Saving figure 617
(204288,)
Saving figure 618
(147456,)
Saving figure 619
(204288,)
Saving figure 620
(147456,)
Saving fig